# 1. Hybrid Model

- 선형 회귀 : 추세에 대한 능력이 뛰어나다, 상호 작용을  학습할 수 없다.
- XGBoost : 상호 작용을 학습 할 수 있다, 추세를 학습 할 수 없다.
- Hybrid Model : 두 모델을 동시에 이용한다.

# **2. Components and Residuals**

- Componets of Series  = trend([3. Time Series_2](https://www.notion.so/3-Time-Series_2-12facc11feb84e5197e63b042de1349e?pvs=21) ) + seasons([3. Time Series_3](https://www.notion.so/3-Time-Series_3-a6f983607fb94dfa863d6fd526f45929?pvs=21))  + cycles([3. TIme Series_4](https://www.notion.so/3-TIme-Series_4-a71a509a4c2440968f388934a9e1d954?pvs=21)) + Error
- Residuals( 잔차 ) = target 과 predictions 의 차이 (정답과 예측의 차이)
    - 잔차에는 모델이 학습하지 못한 부분이 포함되어 있다.

# 3. Hybrid Forecasting with Residuals

- 잔차를 사용한 복합 예측
    
    ```python
    # 1. 첫번째 모델로 학습 및 예측
    model_1.fit(X_train_1, y_train)
    y_pred_1 = model_1.predict(X_train)
    
    # 2. 두번째 모델로 잔차 학습 및 예측
    model_2.fit(X_train_2, y_train - y_pred_1)
    y_pred_2 = model_2.predict(X_train_2)
    
    # 3. 전체 모델의 합
    y_pred = y_pred_1 + y_pred_2
    ```
    

# 4. Hybrids 알고리즘 디자인하기

| 날짜 | Close (종가) |
| --- | --- |
| 2024-01 | 100 |
| 2024-02 | 102 |
| 2024-03 | 101 |
| 2024-04 | 103 |

## 4-1. 피쳐 변환 알고리즘 ( 예시: Linear Regression )

| 날짜 | Close (종가) | index |
| --- | --- | --- |
| 2024-01 | 100 | 0 |
| 2024-02 | 102 | 1 |
| 2024-03 | 101 | 2 |
| 2024-04 | 103 | 3 |
- 순서를 피쳐로 변환하여 target 값이 Close 도 되게 학습한다.
- $y = w x + b$
- $w$ 와  $b$ 를 학습한다.
- 여기서 $x$ ( index ) 는 6이 들어와도 가중치와 곱하고, b 를 더해서 **데이터 범위 밖의 예측이 가능하다.**
- `코드`
    
    ```python
    import pandas as pd
    import seaborn as sns
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import matplotlib_inline.backend_inline
    
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    
    matplotlib_inline.backend_inline.set_matplotlib_formats("retina") # svg, retina, png2x ...
    mpl.style.use("seaborn-v0_8")
    mpl.rcParams.update({"figure.constrained_layout.use": True})
    sns.set_context("paper") 
    sns.set_palette("Set2") 
    sns.set_style("whitegrid") 
    
    plt.rc("font", family = "Malgun Gothic")
    plt.rcParams["axes.unicode_minus"] = False
    ```
    
    ```python
    # 데이터 불러오기
    snp_500 = pd.read_csv('../selfstudy/study_data/snp500_history.csv',
                          parse_dates=['Date']).tail(100)
    close_snp = snp_500[["Date","Close"]].copy()
    ```
    
    ```python
    close_snp["index"] = close_snp.index
    close_snp.set_index("Date", inplace=True)
    ```
    
    ```python
    X = close_snp[["index"]]
    y = close_snp[["Close"]]
    ```
    
    ```python
    x_train, x_test, y_train, y_test = train_test_split(X,y, 
                                                        test_size = 20,
                                                        random_state = 42,
                                                        shuffle=False)
    ```
    
    ```python
    model_1 = LinearRegression()
    model_1.fit(x_train, y_train)
    pred_train = model_1.predict(x_train)
    pred_test = model_1.predict(x_test)
    ```
    
    ```python
    pred_train = pd.Series(pred_train.flatten(), index=y_train.index)
    pred_test = pd.Series(pred_test.flatten(), index=y_test.index)
    ```
    
    ```python
    fig, ax = plt.subplots(1,1, figsize=(6,3))
    
    sns.lineplot(data=y_train,
                 x = "Date",
                 y = "Close",
                 color="tab:blue",
                 label="실제 snp train 값",)
    
    sns.lineplot(data=y_test,
                 x = "Date",
                 y = "Close",
                 color="red",
                 label="실제 snp test 값",)
    
    sns.lineplot(x=y_train.index,
                 y=pred_train,
                 color="tab:green",
                 label="예측할 정답 데이터",)
    
    sns.lineplot(x=y_test.index,
                 y=pred_test,
                 color="tab:orange",
                 linewidth=4,
                 label="회귀 예측 결과",);
    ```
    
- 코드 결과
    
    ![6348cb46-4db5-4414-a61a-3f38f14cb6b1.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/e8f11927-b70c-4524-9227-a3efac08e7aa/dd54cc36-c885-4ad9-8d7a-8dc2a6443706/6348cb46-4db5-4414-a61a-3f38f14cb6b1.png)
    

## 4-2. 타겟 변환 알고리즘 ( 예시: Tree 모델 종류 )

- 조건식 index ≤ 2 일 때 평균 102
- 조건식 index > 2 일 때 평균 103 등
- **예측 값이 데이터 안의 수치를 벗어 날 수 없다.**
- `코드`
    
    ```python
    model_2 = DecisionTreeRegressor(max_depth=2)
    model_2.fit(x_train, y_train)
    pred_train_2 = model_2.predict(x_train)
    pred_test_2 = model_2.predict(x_test)
    ```
    
    ```python
    fig, ax = plt.subplots(1,1, figsize=(12,6))
    
    sns.lineplot(data=y_train,
                 x = "Date",
                 y = "Close",
                 color="tab:blue",
                 label="실제 snp train 값",)
    
    sns.lineplot(data=y_test,
                 x = "Date",
                 y = "Close",
                 color="red",
                 label="실제 snp test 값",)
    
    sns.lineplot(x=y_train.index,
                 y=pred_train_2,
                 color="tab:green",
                 label="예측할 정답 데이터",)
    
    sns.lineplot(x=y_test.index,
                 y=pred_test_2,
                 color="tab:orange",
                 linewidth=4,
                 label="회귀 예측 결과",);
    ```
    
- 코드 결과
    
    ![fdc912d5-83a4-4e81-9740-41cf27eab904.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/e8f11927-b70c-4524-9227-a3efac08e7aa/2144fd71-e027-4666-ae22-16da783642d8/fdc912d5-83a4-4e81-9740-41cf27eab904.png)
    

## 4-3. 위의 두 모델 결합하기

- 4-1 의 Linear Regression 모델은 그대로 사용하고, 4-2 의 모델의 input 데이터를 Resid ( 잔차로 바꾸어 학습시킨다. )
- 4-2 의 Resid 구하기
    
    ```python
    resid = pd.DataFrame(data = pred_train.values,index = pred_train.index,columns= ["Close"])\
        - y_train
    
    fig, ax = plt.subplots(1,1, figsize=(12,6))
    
    sns.lineplot(data=resid,
                 x = "Date",
                 y = "Close",
                 color="tab:blue",
                 label="Residuals",);
    ```
    
    - 잔차 그래프
        
        ![adda5582-78ce-48dd-9137-b7e65b03bbd8.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/e8f11927-b70c-4524-9227-a3efac08e7aa/28f4fdeb-0332-4579-980f-6cf7dc82c0eb/adda5582-78ce-48dd-9137-b7e65b03bbd8.png)
        
- 회귀와 잔차에 대한 예측 결과
    
    ```python
    resid.rename(columns={"Close":"resid"}, inplace=True)
    
    model_3 = DecisionTreeRegressor()
    model_3.fit(x_train.values,resid[["resid"]])
    pred_test_3 = model_3.predict(pd.DataFrame(data=pred_test_2,index=y_test.index,columns=["Close"]) - y_test)
    pred_train_3 = model_3.predict(pd.DataFrame(data=pred_train,index=y_train.index,columns=["Close"]) - y_train)
    ```
    
    ```python
    final_pred_train = pred_train - pred_train_3
    final_pred_test = pred_test - pred_test_3
    ```
    
    ```python
    fig, ax = plt.subplots(1,1, figsize=(12,6))
    
    sns.lineplot(data=y_train,
                 x = "Date",
                 y = "Close",
                 color="tab:blue",
                 label="실제 snp train 값",)
    
    sns.lineplot(data=y_test,
                 x = "Date",
                 y = "Close",
                 color="red",
                 label="실제 snp test 값",)
    
    sns.lineplot(x=y_train.index,
                 y=pred_train,
                 color="tab:green",
                 label="회귀 선",)
    
    sns.lineplot(x=y_test.index,
                 y=pred_test,
                 color="tab:green",
                 label="회귀 선",)
    
    sns.lineplot(x=y_train.index,
                 y=final_pred_train,
                 color="tab:orange",
                 linewidth=4,
                 label="train data 회귀 + 잔차 예측 결과",);
    
    sns.lineplot(x=y_test.index,
                 y=final_pred_test,
                 color="tab:orange",
                 linewidth=4,
                 label="test data 회귀 + 잔차 예측 결과",);
    ```
    
    ![ebdf689c-2b90-468e-b1a3-205c85f28f47.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/e8f11927-b70c-4524-9227-a3efac08e7aa/f139982a-ce00-4706-bd89-d773d7761ec3/ebdf689c-2b90-468e-b1a3-205c85f28f47.png)
    
- 트랜드(다항특성)를 고려하여 잔차와 함께 학습하기

```python
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

matplotlib_inline.backend_inline.set_matplotlib_formats("retina") # svg, retina, png2x ...
mpl.style.use("seaborn-v0_8")
mpl.rcParams.update({"figure.constrained_layout.use": True})
sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

plt.rc("font", family = "Malgun Gothic")
plt.rcParams["axes.unicode_minus"] = False
```

- 그림 그리는 반복문을 줄이기 위한 함수
    
    ```python
    def drawing_graph(X,pt,xt,tt,xt2):
        fig, ax = plt.subplots(1,1, figsize=(12,6))
    
        sns.lineplot(data=X,
                     x = "Date",
                     y = "Close",
                     color="red",
                     label="실제 snp test 값",ax=ax)
    
        sns.lineplot(x=xt.index,
                     y=pt.flatten(),
                     color="tab:green",
                     alpha = 0.8,
                     linewidth = 3,
                     label="예측 데이터",ax=ax)
        sns.lineplot(x=xt2.index,
                     y=tt.flatten(),
                     alpha = 1,
                     color="tab:orange",
                     linewidth = 3,
                     label="예측 데이터",ax=ax)
    
    ```
    
- 데이터 불러오기
    
    ```python
    # 데이터 불러오기
    snp_500 = pd.read_csv('../selfstudy/study_data/snp500_history.csv',
                          parse_dates=['Date']).tail(300)
    close_snp = snp_500[["Date","Close"]].copy()
    ```
    
- 라벨 정하기 및 다항 피쳐 넣기
    
    ```python
    y = close_snp[["Close"]]
    
    # 다항 Trand 피쳐 만들기
    # 테스트 데이터도 묶어서 다항특성을 만들었다.
    # 어차피 그 이후 데이터가 들어와도,
    # 순번에 따라 x^0, x^1, x^2 를 하면 되기떄문에
    dp = DeterministicProcess(
        index=y.index,  # dates from the training data
        constant=True,  # the intercept
        order=2,        # quadratic trend
        drop=True,      # drop terms to avoid collinearity
    )
    
    X = dp.in_sample() 
    ```
    
- 트레인 테스트 분할
    
    ```python
    x_train, x_test, y_train, y_test = train_test_split(X,y, 
                                                        test_size = 0.2,
                                                        shuffle=False)
    ```
    
- trand 의 학습 (2차 다항식)
    
    ```python
    # 1 트렌드 학습하기
    model_trend = LinearRegression()
    model_trend.fit(x_train, y_train)
    trend_pred_train = model_trend.predict(x_train)
    trend_pred_test = model_trend.predict(x_test)
    drawing_graph(snp_500,trend_pred_train,x_train,trend_pred_test,x_test)
    ```
    
    ![b647f4b0-58fb-4beb-bc2f-8f00215c8834.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/e8f11927-b70c-4524-9227-a3efac08e7aa/359a27e6-3843-4b49-a857-3aaac3fbea23/b647f4b0-58fb-4beb-bc2f-8f00215c8834.png)
    
- resid 에 대한 학습 (DecisionTreeRegressor) 을 이용
    
    ```python
    resid_train = y_train - trend_pred_train
    model_tree = DecisionTreeRegressor(max_depth=4)
    model_tree.fit(x_train,resid_train)
    pred_fin_train = model_tree.predict(x_train) + trend_pred_train.reshape(240,)
    pred_fin_test = model_tree.predict(x_test) + trend_pred_test.reshape(60,)
    
    drawing_graph(snp_500,pred_fin_train,x_train,pred_fin_test,x_test)
    ```
    
    ![c28d0880-2032-4219-95f9-f4e449b0c007.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/e8f11927-b70c-4524-9227-a3efac08e7aa/fbe08b3c-e43d-43d9-bb38-866532dbec07/c28d0880-2032-4219-95f9-f4e449b0c007.png)
    
- 이와 같이 우선 trend 에 대한 LinearRegression를 먼저 fit 하고 그 잔차에 대한 값을 결정 트리를 통해 fit 하였다
- 여기서 트리의 max_depth 가 너무 많아지면 오버 핏 하여 일반화 기능이 떨어지기 때문에 실험에 의하여 4정도로 하다
- 특히 LinearRegression 로 선형 적인 부분을 먼저 학습하고
- 그 이후에 비 선형적인 특징은 결정 트리로 학습을 해 보았다.